In [1]:
%cd  G:\Shared drives\INVent\mfk58\Dataset\

#ensure file directory is correct

%ls 

G:\Shared drives\INVent\mfk58\Dataset
 Volume in drive G is Google Drive
 Volume Serial Number is 1983-1116

 Directory of G:\Shared drives\INVent\mfk58\Dataset

05/30/2023  03:50 PM    <DIR>          .
05/31/2023  04:08 PM    <DIR>          ..
05/30/2023  03:50 PM    <DIR>          CCF_Pre_Resampled
05/30/2023  03:53 PM    <DIR>          UH_Pre_Resampled
               0 File(s)              0 bytes
               4 Dir(s)  312,345,845,760 bytes free


In [2]:
%pip install Pillow
%pip install numpy
import glob
#import numpy

import SimpleITK as sitk

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\theev\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\Users\theev\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


In [25]:


#check some basic sanity check information to see if the masks are the same
def hasMask(name):
    files = glob.glob(name+"/*.mha")

    reader = sitk.ImageFileReader()

    if(len(files)<2):
        return False

    reader.SetFileName(files[0])
    reader.LoadPrivateTagsOn()
    reader.ReadImageInformation()

    size1 = reader.GetSize()  
    res1 = reader.GetSpacing() 

    
    reader.SetFileName(files[1])
    reader.LoadPrivateTagsOn()
    reader.ReadImageInformation()

    size2 = reader.GetSize()  
    res2 = reader.GetSpacing()
    
    if not (size1 == size2 and res2 == res2):
        return False


    return True


def axial(name):
    files = glob.glob(name+"/*.mha")
    if("ax" in files[0] and "ax" in files[1]):
        return True
    return False



CCF_count = 0

CCF_missing = [] #missing masks
CCF_list = []

for name in glob.glob('G:/Shared drives/INVent/mfk58/Dataset/CCF_Pre_Resampled/*'):
    
    if not (hasMask(name)):
        CCF_missing.append(name)
        continue
    if axial(name):
        CCF_count +=1
        CCF_list.append(name)
    

print("Total CCF: " + str(CCF_count))
print("Missing masks for CCF:" + str(CCF_missing))




UH_count = 0

UH_missing = [] #missing masks
UH_list = []

for name in glob.glob('G:/Shared drives/INVent/mfk58/Dataset/UH_Pre_Resampled/*'):
    

    if not (hasMask(name)):
        UH_missing.append(name)
        continue
    if axial(name):
        UH_count +=1
        UH_list.append(name)

    

print("Total UH: " + str(UH_count))
print("Missing masks for UH:" + str(UH_missing))

print("Total: " +str(UH_count+CCF_count))
print("Expected: 165")


Total CCF: 66
Missing masks for CCF:[]
Total UH: 99
Missing masks for UH:[]
Total: 165
Expected: 165


Calculating the greater statistics

In [4]:
%pip install pandas
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\theev\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\theev\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [5]:
import pandas as pd
import numpy as np

split = pd.read_excel(r'G:\Shared drives\INVent\mfk58\Dataset-split.xlsx')
split = split.to_numpy();
#print(split[[1,2]])



names = UH_list
adjusted = []
for end in names:
    end = "UH_" + end.rsplit("\\",1)[-1] + "_pre_ax_raw_resampled"
    adjusted.append(end)

names = CCF_list
for end in names:
    end = "CCF_" + end.rsplit("\\",1)[-1] + "_pre_ax_raw_resampled"
    adjusted.append(end)


#these files do not match the general naming scheme, but they are present
adjusted = list(map(lambda x: x.replace("CCF_RectalCA_011_pre_ax_raw_resampled", "CCF_RectalCA_011_pre_ax_resampled"), adjusted))
adjusted = list(map(lambda x: x.replace("CCF_RectalCA_060_pre_ax_raw_resampled", "CCF_RectalCA_060_pre_ax_resampled"), adjusted))

print("Files that are not in the splitting list")
for name in adjusted:
    if not(np.isin(name,split)):
        print(name)


Files that are not in the splitting list


In [6]:

x = split[:,[0,2]].flatten()
x = np.append(x, split[0:10,3])
x.sort()
#print(x)

count = 0
for name in x:

    if name.split('_',1)[0] == "UH":
        break
    count += 1
    
print("Validation set CCF: " + str(count))
print("Validation set UH: " + str(len(x)-count))

print("Training set CCF: " + str(CCF_count-count))
print("Training set UH: " + str(UH_count-len(x)+count))




Validation set CCF: 47
Validation set UH: 67
Training set CCF: 19
Training set UH: 32


In [ ]:


"""
###THIS IS A REALLY TERRIBLE IMPLEMENTATION - extremely inefficient
comList = CCF_list + UH_list
#print(comList)


maxWidth = 0
madDepth = 0
maxHeight = 0
maxVol = 0

for name in comList:
    files = glob.glob(name+"/*")
    f1 = sitk.ReadImage(files[0], imageIO="MetaImageIO")
    f2 = sitk.ReadImage(files[1], imageIO="MetaImageIO")

    w1 = f1.GetWidth()
    w2 = f2.GetWidth()

    h1 = f1.getHeight()
    h2 = f2.GetHeight()

    d1 = f1.GetDepth()
    d2 = f2.GetDepth()


    print("running " +name)
    


    #del f1, f2, files #needed this to not make the memory overflow

"""


In [43]:


#find maximum values
import pathlib
import SimpleITK as sitk

comList = CCF_list + UH_list
#print(comList)

dimensions = [0,0,0,0,0] #x,y,z, x-y area, vol
dfiles = ["","","","",""]
max = [0,0,0,0,0]


reader = sitk.ImageFileReader()




print (len(comList))
for name in comList:
    #print("running " +name)
    files = glob.glob(name+"/*.mha")
    if not ("ax" in files[0] and "ax" in files[1]):
        continue
    for file in files:
        reader.SetFileName(file)
        reader.LoadPrivateTagsOn()
        reader.ReadImageInformation()

        size = reader.GetSize()  
        xres, yres, zres = reader.GetSpacing() 
        size = list(size)
        size.append(np.prod(size[0:2]))
        size.append(np.prod(size[0:3]))

        for dim in range(len(dimensions)):
            if size[dim] > dimensions[dim]:
                dimensions[dim] = size[dim]
                dfiles[dim] = name
                max[dim] = size

            if not (xres == 1 and yres == 1 and zres == 1):
                print("resolution is incorrect for: " + name)


table = [dimensions,dfiles,max]
print(table)
        






    
    



165
[[320, 300, 360, 90000, 21276000], ['G:/Shared drives/INVent/mfk58/Dataset/CCF_Pre_Resampled\\RectalCA_027', 'G:/Shared drives/INVent/mfk58/Dataset/CCF_Pre_Resampled\\RectalCA_011', 'G:/Shared drives/INVent/mfk58/Dataset/CCF_Pre_Resampled\\RectalCA_037', 'G:/Shared drives/INVent/mfk58/Dataset/CCF_Pre_Resampled\\RectalCA_011', 'G:/Shared drives/INVent/mfk58/Dataset/CCF_Pre_Resampled\\RectalCA_037'], [[320, 210, 99, 67200, 6652800], [300, 300, 144, 90000, 12960000], [300, 197, 360, 59100, 21276000], [300, 300, 144, 90000, 12960000], [300, 197, 360, 59100, 21276000]]]


In [45]:


#find minimum values



#based on the maximums obtained from the previous block - these should be large enough
dimensions = [10000,10000,10000,1000000,31276000] #x,y,z, x-y area, vol
dfiles = ["","","","",""]
max = [0,0,0,0,0]


reader = sitk.ImageFileReader()




print (len(comList))
for name in comList:
    #print("running " +name)
    files = glob.glob(name+"/*.mha")
    if not ("ax" in files[0] and "ax" in files[1]):
        continue
    for file in files:
        reader.SetFileName(file)
        reader.LoadPrivateTagsOn()
        reader.ReadImageInformation()

        size = reader.GetSize()  
        xres, yres, zres = reader.GetSpacing() 
        size = list(size)
        size.append(np.prod(size[0:2]))
        size.append(np.prod(size[0:3]))

        for dim in range(len(dimensions)):
            if size[dim] < dimensions[dim]:
                dimensions[dim] = size[dim]
                dfiles[dim] = name
                max[dim] = size

            if not (xres == 1 and yres == 1 and zres == 1):
                print("resolution is incorrect for: " + name)


mintable = [dimensions,dfiles,max]
print(mintable)
        






    
    



165
[[150, 135, 60, 22500, 1920000], ['G:/Shared drives/INVent/mfk58/Dataset/CCF_Pre_Resampled\\RectalCA_039', 'G:/Shared drives/INVent/mfk58/Dataset/CCF_Pre_Resampled\\RectalCA_124', 'G:/Shared drives/INVent/mfk58/Dataset/CCF_Pre_Resampled\\RectalCA_098', 'G:/Shared drives/INVent/mfk58/Dataset/CCF_Pre_Resampled\\RectalCA_039', 'G:/Shared drives/INVent/mfk58/Dataset/UH_Pre_Resampled\\RectalCA_038'], [[150, 150, 200, 22500, 4500000], [180, 135, 173, 24300, 4203900], [230, 230, 60, 52900, 3174000], [150, 150, 200, 22500, 4500000], [160, 160, 75, 25600, 1920000]]]
